In [ ]:
!pip install langchain-groq

In [ ]:
from google.colab import userdata
# LLM with function call
from langchain_groq import ChatGroq
import os
groq_api_key=userdata.get('groq_api_key')
os.environ["GROQ_API_KEY"]=groq_api_key
from langchain_groq import ChatGroq

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name='llama-3.3-70b-versatile',
    temperature=0.1
)

In [ ]:
llm.invoke("Hello, how are you today?")

AIMessage(content="Hello. I'm doing well, thanks for asking. I'm a large language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How about you? How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 42, 'total_tokens': 104, 'completion_time': 0.225454545, 'prompt_time': 0.004666804, 'queue_time': 1.46868053, 'total_time': 0.230121349}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ca0059abb', 'finish_reason': 'stop', 'logprobs': None}, id='run-b5b465c0-c135-4428-b426-07e46c93f588-0', usage_metadata={'input_tokens': 42, 'output_tokens': 62, 'total_tokens': 104})

In [ ]:
llm.invoke("What is the stock price of Apple?")

AIMessage(content="I'm not able to provide real-time stock prices. However, I can suggest some ways for you to find the current stock price of Apple:\n\n1. Check online financial websites: You can visit websites like Yahoo Finance, Google Finance, or Bloomberg to get the current stock price of Apple.\n2. Use a stock market app: You can download a stock market app like Robinhood, Fidelity, or E\\*TRADE to get real-time stock prices.\n3. Visit Apple's investor relations website: Apple's investor relations website provides up-to-date information on the company's stock price, as well as other financial information.\n\nPlease note that stock prices can fluctuate rapidly and may be different from the time I provided this response. For the most accurate and up-to-date information, I recommend checking a reliable financial source.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 166, 'prompt_tokens': 43, 'total_tokens': 209, 'completion_time': 0.603636364, 'promp

## Connecting LLMs to Financial Data

Yahoo Finance API

In [ ]:
pip install yfinance

In [ ]:
import yfinance as yf
from langchain.agents import AgentType, Tool, initialize_agent

# Initialize the LLM
llm = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name='llama-3.3-70b-versatile',
    temperature=0.1
)

## Define the function to get the stock price
def get_stock_price(symbol):
    stock = yf.Ticker(symbol)
    today_data = stock.history(period='1d')
    return f"The current stock price of {symbol} is {today_data['Close'].iloc[-1]:.2f}"

## Create a tool for the agent to use
tools = [
    Tool(
        name="Stock Price",
        func=get_stock_price,
        description="Use this tool to get the current stock price of a given stock symbol. The input to this tool should be the stock symbol."
    )
]

## Initialize the agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

response=agent.invoke("What is the stock price of Apple?")
print(response)


<ipython-input-7-06078e22c772>:27: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
Thought: To find the current stock price of Apple, I need to use the Stock Price tool. I know that the stock symbol for Apple is AAPL. 

Action: Stock Price
Action Input: AAPL
Observation: The current stock price of AAPL is 232.80
Thought:Thought: I now know the final answer
Final Answer: The current stock price of Apple is $232.80.

> Finished chain.
{'input': 'What is the stock price of Apple?', 'output': 'The current stock price of Apple is $232.80.'}


In [ ]:
from langchain_core.tools import tool, StructuredTool
from datetime import date

@tool
def company_information(ticker: str) -> dict:
    """Use this tool to retrieve company information like address, industry, sector, company officers, business summary, website,
       marketCap, current price, ebitda, total debt, total revenue, debt-to-equity, etc."""

    ticker_obj = yf.Ticker(ticker)
    ticker_info = ticker_obj.get_info()

    return ticker_info

@tool
def last_dividend_and_earnings_date(ticker: str) -> dict:
    """
    Use this tool to retrieve company's last dividend date and earnings release dates.
    It does not provide information about historical dividend yields.
    """
    ticker_obj = yf.Ticker(ticker)

    return ticker_obj.get_calendar()

@tool
def summary_of_mutual_fund_holders(ticker: str) -> dict:
    """
    Use this tool to retrieve company's top mutual fund holders.
    It also returns their percentage of share, stock count and value of holdings.
    """
    ticker_obj = yf.Ticker(ticker)
    mf_holders = ticker_obj.get_mutualfund_holders()

    return mf_holders.to_dict(orient="records")

@tool
def summary_of_institutional_holders(ticker: str) -> dict:
    """
    Use this tool to retrieve company's top institutional holders.
    It also returns their percentage of share, stock count and value of holdings.
    """
    ticker_obj = yf.Ticker(ticker)
    inst_holders = ticker_obj.get_institutional_holders()

    return inst_holders.to_dict(orient="records")

@tool
def stock_grade_updrages_downgrades(ticker: str) -> dict:
    """
    Use this to retrieve grade ratings upgrades and downgrades details of particular stock.
    It'll provide name of firms along with 'To Grade' and 'From Grade' details. Grade date is also provided.
    """
    ticker_obj = yf.Ticker(ticker)

    curr_year = date.today().year

    upgrades_downgrades = ticker_obj.get_upgrades_downgrades()
    upgrades_downgrades = upgrades_downgrades.loc[upgrades_downgrades.index > f"{curr_year}-01-01"]
    upgrades_downgrades = upgrades_downgrades[upgrades_downgrades["Action"].isin(["up", "down"])]

    return upgrades_downgrades.to_dict(orient="records")

@tool
def stock_splits_history(ticker: str) -> dict:
    """
    Use this tool to retrieve company's historical stock splits data.
    """
    ticker_obj = yf.Ticker(ticker)
    hist_splits = ticker_obj.get_splits()

    return hist_splits.to_dict()

@tool
def stock_news(ticker: str) -> dict:
    """
    Use this to retrieve latest news articles discussing particular stock ticker.
    """
    ticker_obj = yf.Ticker(ticker)

    return ticker_obj.get_news()

In [ ]:
from langchain.agents import AgentExecutor, initialize_agent, AgentType
from langchain.agents import Tool

# Assuming you've already defined your tools and LLM:
tools = [
    Tool(
        name="company_information",
        func=company_information,
        description="Use this tool to retrieve company information like address, industry, sector, company officers, business summary, website, marketCap, current price, ebitda, total debt, total revenue, debt-to-equity, etc.",
    ),
    Tool(
        name="last_dividend_and_earnings_date",
        func=last_dividend_and_earnings_date,
        description="Use this tool to retrieve company's last dividend date and earnings release dates. It does not provide information about historical dividend yields.",
    ),
    Tool(
        name="stock_splits_history",
        func=stock_splits_history,
        description="Use this tool to retrieve company's historical stock splits data.",
    ),
    Tool(
        name="summary_of_mutual_fund_holders",
        func=summary_of_mutual_fund_holders,
        description="Use this tool to retrieve company's top mutual fund holders. It also returns their percentage of share, stock count and value of holdings.",
    ),
    Tool(
        name="summary_of_institutional_holders",
        func=summary_of_institutional_holders,
        description="Use this tool to retrieve company's top institutional holders. It also returns their percentage of share, stock count and value of holdings.",
    ),
    Tool(
        name="stock_grade_updrages_downgrades",
        func=stock_grade_updrages_downgrades,
        description="Use this to retrieve grade ratings upgrades and downgrades details of particular stock. It'll provide name of firms along with 'To Grade' and 'From Grade' details. Grade date is also provided.",
    ),
    Tool(
        name="stock_news",
        func=stock_news,
        description="Use this to retrieve latest news articles discussing particular stock ticker.",
    ),
    Tool(
        name="Stock Price",
        func=get_stock_price,
        description="Use this tool to get the current stock price of a given stock symbol. The input to this tool should be the stock symbol."
    )
]

# Create the financial agent using initialize_agent
finance_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Execute the agent
query = "Give me Apple's company information and its current stock price."
response = finance_agent.run(query)

print(response)



> Entering new AgentExecutor chain...


<ipython-input-9-6a1aa32b9cd0>:58: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = finance_agent.run(query)


To get Apple's company information and its current stock price, I need to use the company_information tool and the Stock Price tool. 

Action: company_information
Action Input: Apple

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/APPLE?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=APPLE&crumb=c3f2Flytn6j



Observation: {'trailingPegRatio': None}
Thought:It seems like the observation from the company_information tool is incomplete. Let me try again to get a more complete picture of Apple's company information.

Action: company_information
Action Input: Apple

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/APPLE?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=APPLE&crumb=c3f2Flytn6j



Observation: {'trailingPegRatio': None}
Thought:It seems like the observation from the company_information tool is still incomplete. Let me try again to get a more complete picture of Apple's company information.

Action: company_information
Action Input: Apple

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/APPLE?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=APPLE&crumb=c3f2Flytn6j



Observation: {'trailingPegRatio': None}
Thought:It seems like the observation from the company_information tool is still incomplete. However, I should also try to get the current stock price of Apple.

Action: Stock Price
Action Input: AAPL
Observation: The current stock price of AAPL is 232.80
Thought:I now know the final answer
Final Answer: The current stock price of Apple (AAPL) is 232.80. However, I was unable to retrieve a complete picture of Apple's company information as the company_information tool returned incomplete observations.

> Finished chain.
The current stock price of Apple (AAPL) is 232.80. However, I was unable to retrieve a complete picture of Apple's company information as the company_information tool returned incomplete observations.


In [ ]:
query = "What are ebitda, total debt, total revenue and debt-to-equity for Walmart stock?"
response = finance_agent.run(query)

print(response)



> Entering new AgentExecutor chain...
To find the ebitda, total debt, total revenue, and debt-to-equity for Walmart stock, I need to use the company_information tool. This tool can provide various financial metrics for a given company.

Action: company_information
Action Input: Walmart

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/WALMART?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=WALMART&crumb=c3f2Flytn6j



Observation: {'trailingPegRatio': None}
Thought:It seems like the observation is not providing the required information. Let me try again with a different approach.

Action: company_information
Action Input: WMT
Observation: {'address1': '702 South West 8th Street', 'city': 'Bentonville', 'state': 'AR', 'zip': '72716', 'country': 'United States', 'phone': '479-273-4000', 'website': 'https://corporate.walmart.com', 'industry': 'Discount Stores', 'industryKey': 'discount-stores', 'industryDisp': 'Discount Stores', 'sector': 'Consumer Defensive', 'sectorKey': 'consumer-defensive', 'sectorDisp': 'Consumer Defensive', 'longBusinessSummary': "Walmart Inc. engages in the operation of retail, wholesale, other units, and eCommerce worldwide. The company operates through three segments: Walmart U.S., Walmart International, and Sam's Club. It operates supercenters, supermarkets, hypermarkets, warehouse clubs, cash and carry stores, and discount stores under Walmart and Walmart Neighborhood Marke

In [ ]:
# Create a dummy portfolio of 5 stocks
portfolio = {
    'AAPL': 50,  # Apple
    'MSFT': 30,  # Microsoft
    'GOOGL': 20, # Alphabet (Google)
    'AMZN': 15,  # Amazon
    'NVDA': 25   # NVIDIA
}

# Function to analyze a single stock
def analyze_stock(ticker: str, shares: int):
    # Added a try-except block to handle potential API errors
    try:
        response = finance_agent.run(f"Provide a brief analysis of {ticker} including current price, P/E ratio, and recent performance.")
        return f"{ticker} ({shares} shares): {response}"
    except Exception as e:
        print(f"Error analyzing {ticker}: {e}")  # Print error message for debugging
        return f"{ticker} ({shares} shares): Analysis failed."

# Function to analyze the entire portfolio
def analyze_portfolio(portfolio: dict):
    analysis = []
    total_value = 0

    for ticker, shares in portfolio.items():
        stock_analysis = analyze_stock(ticker, shares)
        analysis.append(stock_analysis)

        # Get current price to calculate portfolio value
        # Added a try-except block to handle potential API errors
        try:
            price_response = finance_agent.run(f"What is the current stock price of {ticker}?")
            price = float(price_response.split('$')[1].split()[0])
            total_value += price * shares
        except Exception as e:
            print(f"Could not parse price for {ticker}: {e}") # Print error message for debugging

    return analysis, total_value

# Analyze the portfolio
print("Analyzing portfolio...")
stock_analyses, portfolio_value = analyze_portfolio(portfolio)

# Print the results
print("\nPortfolio Analysis:")
for analysis in stock_analyses:
    print(analysis)
    print("-" * 50)

Analyzing portfolio...


> Entering new AgentExecutor chain...
To provide a brief analysis of AAPL, I need to gather information about the company's current price, financial metrics such as the P/E ratio, and its recent performance. 

Action: Stock Price
Action Input: AAPL
Observation: The current stock price of AAPL is 232.80
Thought:To calculate the P/E ratio, I need to know the company's current earnings per share (EPS). The P/E ratio is calculated by dividing the current stock price by the EPS. I can use the company_information tool to retrieve the EPS and other relevant financial metrics.

Action: company_information
Action Input: AAPL
Observation: {'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '(408) 996-1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'industryKey': 'consumer-electronics', 'industryDisp': 'Consumer Electronics', 'sector': 'Technology', 'sectorKey': 'technol

In [ ]:
pip install phidata

In [ ]:
pip install duckduckgo-search

In [ ]:
# from phi.agent import Agent
import phi
from phi.tools.duckduckgo import DuckDuckGo
from phi.tools.yfinance import YFinanceTools
from langchain_groq import ChatGroq
import os

# Set the GROQ API key
groq_api_key = os.getenv("GROQ_API_KEY")  # Make sure to set this in your environment

# Initialize the LLM using Groq
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name='llama-3.3-70b-versatile',
    temperature=0.1
)

# Import Agent from phi.agent
from phi.agent import Agent # Importing the Agent class

# Web Agent using DuckDuckGo
web_agent = Agent(
    name="Web Agent",
    role="Search the web for information",
    model=llm,
    tools=[DuckDuckGo()],
    markdown=True,
    show_tool_calls=True,
)

# Finance Agent using YFinanceTools
finance_agent = Agent(
    name="Finance Agent",
    role="Get financial data",
    model=llm,
    tools=[
        YFinanceTools(
            stock_price=True,
            analyst_recommendations=True,
            company_info=True,
            company_news=True
        )
    ],
    instructions=["Always use tables to display data"],
    markdown=True,
    show_tool_calls=True,
)

# Example queries
web_response = web_agent.run("Find the latest news about AI technology.")
finance_response = finance_agent.run("Get the current stock price and analyst recommendations for Apple (AAPL).")

# Display responses
print("Web Agent Response:\n", web_response)
print("\nFinance Agent Response:\n", finance_response)

ValidationError: 1 validation error for Agent
model
  Input should be a valid dictionary or instance of Model [type=model_type, input_value=ChatGroq(client=<groq.res...SecretStr('**********')), input_type=ChatGroq]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type